# Linear Regression Health Costs Calculator

In this project, the goal was to predict healthcare costs using a regression algorithm.

I also will use PyTorch for this notebook, but you can always see the original solution with TensorFlow below.

To see the original solution refer to [here]("https://colab.research.google.com/drive/1gO8bcKY4yxBfVVyEK3bBUjEeOZOXBnAZ?usp=sharing") and more informations [here]("https://www.freecodecamp.org/learn/machine-learning-with-python/machine-learning-with-python-projects/linear-regression-health-costs-calculator")

---

In [1]:
# Import libraries. You may or may not use all of these.
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split 

import torch
from torch import nn

c:\Users\Lucas\AppData\Local\Programs\Python\data-science\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import os
from pathlib import Path

# Import data
data_path = Path("data")
insurance_path = data_path / "insurance"

if insurance_path.is_dir():
    print(f"{insurance_path} already exists, skipping")

!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()